In [11]:
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")


In [12]:
# custome memory cell with layer normalization
class LNMemory(tf.keras.layers.Layer):
    def __init__(self, units, activation= "tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = tf.keras.layers.SimpleRNNCell(units, activation= None)
        self.ln_layer = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)
    
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.ln_layer(outputs))
        return norm_outputs, [norm_outputs] # next rnn step expects tuple or list input

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape= [None, 1]),
    tf.keras.layers.RNN(LNMemory(10), return_sequences= True), # input_shape ?
    tf.keras.layers.RNN(LNMemory(10), return_sequences= True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(10))
])